# WMD distance implementation using (gensim + google word2vec embeddings)

In [18]:
from nltk.corpus import stopwords
import pandas as pd
import os
from time import time
import numpy as np

In [186]:
train = pd.read_csv('data/train.csv')
# test = pd.read_csv('data/test.csv')

In [187]:
len(train)

404290

In [188]:
int(len(train)*0.8)

323432

## Split training dataset to (80% train + 20% test)

In [189]:
# Since we dont have testing labels, lets split train dataset to (train + test) split
train_len = int(len(train)*0.8)
test = train[train_len:]
train = train[:train_len]

In [190]:
len(train)

323432

In [191]:
len(test)

80858

In [192]:
# Access first row
train.iloc[0]

id                                                              0
qid1                                                            1
qid2                                                            2
question1       What is the step by step guide to invest in sh...
question2       What is the step by step guide to invest in sh...
is_duplicate                                                    0
Name: 0, dtype: object

In [193]:
# Access column-question1, row-0
train['question1'].iloc[0]

'What is the step by step guide to invest in share market in india?'

In [194]:
stop_words = stopwords.words('english')

In [195]:
type(train)

pandas.core.frame.DataFrame

In [196]:
for idx, row in train.head().iterrows():
    print(row)

id                                                              0
qid1                                                            1
qid2                                                            2
question1       What is the step by step guide to invest in sh...
question2       What is the step by step guide to invest in sh...
is_duplicate                                                    0
Name: 0, dtype: object
id                                                              1
qid1                                                            3
qid2                                                            4
question1       What is the story of Kohinoor (Koh-i-Noor) Dia...
question2       What would happen if the Indian government sto...
is_duplicate                                                    0
Name: 1, dtype: object
id                                                              2
qid1                                                            5
qid2                          

In [197]:
for idx, row in train.head().iterrows():
    s1 = row['question1']
    print(s1)

What is the step by step guide to invest in share market in india?
What is the story of Kohinoor (Koh-i-Noor) Diamond?
How can I increase the speed of my internet connection while using a VPN?
Why am I mentally very lonely? How can I solve it?
Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?


## Preprocessing

In [198]:
for idx, row in train.head(n=1).iterrows():
    s1 = row['question1'].lower().split()
    s2= row['question2'].lower().split()
    
    s1 = [i for i in s1 if i not in stop_words]
    s2= [i for i in s2 if i not in stop_words]
    print(s1, s2)

['step', 'step', 'guide', 'invest', 'share', 'market', 'india?'] ['step', 'step', 'guide', 'invest', 'share', 'market?']


In [199]:
'''
we need some word embeddings first of all. You could train a word2vec model on some corpus, 
but we will start by downloading some pre-trained word2vec embeddings. Download the GoogleNews-vectors-negative300.bin.gz
embeddings
'''
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

if not os.path.exists('data/w2v_googlenews/GoogleNews-vectors-negative300.bin.gz'):
    raise ValueError("SKIP: You need to download the google news model")

In [200]:
start = time()
model = KeyedVectors.load_word2vec_format('data/w2v_googlenews/GoogleNews-vectors-negative300.bin.gz', binary=True)
print('Cell took %.2f seconds to run.' % (time() - start))

Cell took 138.46 seconds to run.


## Word Mover's Distance basics
WMD is a method that allows us to assess the "distance" between two documents in a meaningful way, 
even when they have no words in common. It uses word2vec vector embeddings of words. It been shown to outperform many of the
state-of-the-art methods in k-nearest neighbors classification.  The sentences have no words in common, but by matching the
relevant words, WMD is able to accurately measure the (dis)similarity between the two sentences. The method also uses the 
bag-of-words representation of the documents (simply put, the word's frequencies in the documents)

The intution behind the method is that we find the minimum "traveling distance" between documents, in other words the most efficient way to "move" the distribution of document 1 to the distribution of document 2.
 
WMD is illustrated below for two very similar sentences


![title](Images/wmd.png)

This method was introduced in the article "From Word Embeddings To Document Distances" by Matt Kusner et al. [blue_text](http://proceedings.mlr.press/v37/kusnerb15.pdf). It is inspired by the "Earth Mover's Distance", and employs a solver of the "transportation problem".

In this tutorial, we will learn how to use Gensim's WMD functionality, which consists of the wmdistance method for distance computation, and the WmdSimilarity class for corpus based similarity queries.

In [201]:
# So let's compute WMD using the wmdistance method.
for idx, row in train.head(n=10).iterrows():
    q1 = row['question1']
    q2 = row['question2']
    print('\n',q1)
    print(q2)
    distance = model.wmdistance(q1, q2)
    print('distance = %.4f' % distance)


 What is the step by step guide to invest in share market in india?
What is the step by step guide to invest in share market?
distance = 0.2949

 What is the story of Kohinoor (Koh-i-Noor) Diamond?
What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?
distance = 0.9528

 How can I increase the speed of my internet connection while using a VPN?
How can Internet speed be increased by hacking through DNS?
distance = 0.6550

 Why am I mentally very lonely? How can I solve it?
Find the remainder when [math]23^{24}[/math] is divided by 24,23?
distance = 1.4342

 Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?
Which fish would survive in salt water?
distance = 0.8907

 Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?
distance = 0.4784

 Should I buy tiago?
What keeps childern active and far from phone 

## Calculate WMD distance from gensim model

In [202]:
# Generate x_train and y_train
x_train = [] # distances
y_train = [] # is_duplicate
for idx, row in train.iterrows():
    q1 = row['question1']
    q2 = row['question2']
    try:
        distance = model.wmdistance(q1, q2)
        x_train.append(distance)
        y_train.append(row['is_duplicate'])
    except:
        continue    

In [203]:
sorted(set(x_train), reverse=True)[:5]

[inf,
 3.697866162030013,
 3.4708841502605083,
 3.427613456417332,
 3.335370726894855]

In [204]:
# Limitaion of gensim.models.Word2Vec.wmdistance
# Note that if one of the documents have no words that exist in the Word2Vec vocab, float(‘inf’) (i.e. infinity) will be returned.
# Hence replacing infinity with large distance
for idx, val in enumerate(x_train):
    if(np.isinf(val)):
        x_train[idx] = 5 #large number

In [205]:
sorted(set(x_train), reverse=True)[:5]

[5,
 3.697866162030013,
 3.4708841502605083,
 3.427613456417332,
 3.335370726894855]

## Training - Logistic Regression

#### how about we try linear regression on wmd-distance

In [206]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()

# Reshape your data either using array.reshape(-1, 1) if your data has a single feature 
x_train = np.array(x_train).reshape(-1, 1)
y_train = np.array(y_train).reshape(-1, 1)

logmodel.fit(x_train,y_train)
# regr.fit([x_train], [y_train])

C:\Users\VinayBysani\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\VinayBysani\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

## Testing

In [207]:
# Testing on training dataset from last, since we dont have access to testing labels
x_test = [] # distances
y_test = [] # is_duplicate
for idx, row in test.iterrows():
    q1 = row['question1']
    q2 = row['question2']
    try:
        distance = model.wmdistance(q1, q2)
        x_test.append(distance)
        y_test.append(row['is_duplicate'])
    except:
        continue    

In [208]:
x_train[:5]

array([[0.29490848],
       [0.95277931],
       [0.65501835],
       [1.43415995],
       [0.89073397]])

In [209]:
x_test[:5]

[1.0716027899104503,
 0.6324009618218077,
 0.6573009572431007,
 0.6319357305656796,
 0.4830612627582875]

In [210]:
# Limitaion of gensim.models.Word2Vec.wmdistance
# Note that if one of the documents have no words that exist in the Word2Vec vocab, float(‘inf’) (i.e. infinity) will be returned.
# Hence replacing infinity with large distance
for idx, val in enumerate(x_test):
    if(np.isinf(val)):
        x_test[idx] = 5 #large number

In [211]:
# Reshape your data either using array.reshape(-1, 1) if your data has a single feature 
x_test = np.array(x_test).reshape(-1, 1)
# y_train = np.array(y_train).reshape(-1, 1)

predicted = logmodel.predict(x_test)

In [212]:
len(predicted)

80857

In [217]:
len(test)

80858

In [213]:
predicted[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [214]:
y_test[:10]

[0, 1, 0, 1, 1, 1, 0, 0, 0, 1]

In [215]:
# Not going beyond 65% accuracy. We can get 50% accuracy in classification by flipping coin as well. So 65% is pretty dumb
logmodel.score(x_test, y_test)

0.658681375762148

In [216]:

# Validates entire training set
correct = 0
for i in range(len(test)):
    if(predicted[i] == y_test[i]):
        correct += 1
print(correct/len(test)*100)

IndexError: index 80857 is out of bounds for axis 0 with size 80857